In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import numpy as np
import pandas as pd
import torch
import pickle
import time
import os
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from functools import partial
from tqdm.auto import tqdm
import statsmodels.api as sm
from multiprocessing import Pool

from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList


In [ ]:
### framework constants:
model_type = 'logistic_regression'
model_version = 'log_reg_sm'
test_name = 'train_log_reg_abby'
pheno_method = 'Abby' # Paul, Abby
number_test = 6
tryout = False # True if we are ding a tryout, False otherwise 
### data constants:
CHR = 1
SNP = 'rs673604'

#analysis constant
p_value_threshold = 1e-4
option = 'whole'

In [ ]:
path = '/gpfs/commons/groups/gursoy_lab/mstoll/codes/'

#check test name
model_dir = path + f'logs/SNPS/{str(CHR)}/{SNP}/{model_type}/{model_version}/{pheno_method}'
#check number tests
test_name_with_infos = str(number_test) + '_' + test_name + 'tryout'*tryout
test_dir = f'{model_dir}/{test_name_with_infos}/'
log_data_dir = f'{test_dir}/data/'
log_res_dir = f'{test_dir}/res/'
log_slurm_outputs_dir = f'{test_dir}/Slurm/Outputs/'
log_slurm_errors_dir = f'{test_dir}/Slurm/Errors/'


log_data_path_pickle = f'{test_dir}/data/{test_name}_{option}.pkl'


log_res_path_pickle = f'{test_dir}/res/{test_name}_{option}.pkl'
log_slurm_outputs_path = f'{test_dir}/Slurm/Outputs/{test_name}.pth'
log_slurm_error_path = f'{test_dir}/Slurm/Errors/{test_name}.pth'





In [ ]:
with open(log_data_path_pickle, 'rb') as file:
    tree_data_dic  = pickle.load(file)


In [ ]:
with open(log_res_path_pickle, 'rb') as file:
    dic_res = pickle.load(file)

In [ ]:
pheno_data_train = tree_data_dic['pheno_data_train']
pheno_data_test = tree_data_dic['pheno_data_test']
label_data_train = tree_data_dic['label_data_train']
label_data_test = tree_data_dic['label_data_test']

In [ ]:
models = dic_res['models']
p_values = dic_res['p_values']
coeffs = dic_res['coeffs']

In [ ]:
models

In [ ]:
models = np.array(models)
indices_not_none = np.where(models != None)
nb_phenos_available = len(indices_not_none)

In [ ]:
pheno_data_test.shape

In [ ]:
for indice_pheno_test in indices_not_none:
    pheno_data_test = pheno_data_test[:, indice_pheno_test]
    model = models[indice_pheno_test]
    predictions = model.predict(pheno_data_test)
    label_predictions = (predictions>0.5).astype(int)
    error = np.sqrt(np.sum(label_data_test - predictions))**2 / len(predictions)

In [ ]:
error

In [ ]:
indices_p_values_meaningful = np.where(p_values < p_value_threshold)

In [ ]:
indices_p_values_meaningful